In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

C:\Users\ACER\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
test = pd.read_csv('../data/test.csv', dtype={'ID': 'int32', 'shop_id': 'int32', 
                                                  'item_id': 'int32'})
item_categories = pd.read_csv('../data/item_categories.csv', 
                              dtype={'item_category_name': 'str', 'item_category_id': 'int32'})
items = pd.read_csv('../data/items.csv', dtype={'item_name': 'str', 'item_id': 'int32', 
                                                 'item_category_id': 'int32'})
shops = pd.read_csv('../data/shops.csv', dtype={'shop_name': 'str', 'shop_id': 'int32'})
sales = pd.read_csv('../data/sales_train.csv', parse_dates=['date'], 
                    dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32', 
                          'item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'})

In [50]:
# train = sales.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(item_categories, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)

In [3]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.000000,1
1,2013-03-01,0,25,2552,899.000000,1
2,2013-05-01,0,25,2552,899.000000,-1
3,2013-06-01,0,25,2554,1709.050049,1
4,2013-01-15,0,25,2555,1099.000000,1


In [4]:
sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.906234e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.726440e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [5]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [15]:
sales['date'] = pd.to_datetime(sales['date'], format="%Y %m %d")

In [16]:
sales['month'] = sales['date'].dt.month

In [17]:
sales['year'] = sales['date'].dt.year

In [18]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year
0,2013-02-01,0,59,22154,999.000000,1,2,2013
1,2013-03-01,0,25,2552,899.000000,1,3,2013
2,2013-05-01,0,25,2552,899.000000,-1,5,2013
3,2013-06-01,0,25,2554,1709.050049,1,6,2013
4,2013-01-15,0,25,2555,1099.000000,1,1,2013


In [20]:
sales = sales.drop('date', axis=1)

In [28]:
temp = sales.groupby([x for x in sales.columns if x not in ['item_cnt_day']], as_index=False)['item_cnt_day'].sum()

In [29]:
temp.columns = ["date_block_num", "shop_id", "item_id", "item_price", "month", "year", "item_cnt_month"]

In [30]:
temp.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month
0,0,0,32,221.0,1,2013,4
1,0,0,32,221.0,3,2013,2
2,0,0,33,347.0,1,2013,2
3,0,0,33,347.0,3,2013,1
4,0,0,35,247.0,1,2013,1


In [31]:
shop_item_mean = temp[["shop_id", "item_id", "item_cnt_month"]].groupby(["shop_id", "item_id"], as_index=False)['item_cnt_month'].mean()

In [34]:
shop_item_mean.columns = ["shop_id", "item_id", "item_cnt_mean"]

In [35]:
shop_item_mean.head()

,shop_id,item_id,item_cnt_mean
0,0,30,31.000000
1,0,31,11.000000
2,0,32,2.666667
3,0,33,1.500000
4,0,35,1.875000


In [36]:
train = pd.merge(temp, shop_item_mean, how='left', on=['shop_id', 'item_id'])

train.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month,item_cnt_mean
0,0,0,32,221.0,1,2013,4,2.666667
1,0,0,32,221.0,3,2013,2,2.666667
2,0,0,33,347.0,1,2013,2,1.500000
3,0,0,33,347.0,3,2013,1,1.500000
4,0,0,35,247.0,1,2013,1,1.875000


In [37]:
shop_pre_month = train[train['date_block_num'] == 33][['shop_id', 'item_id', 'item_cnt_month']]
shop_pre_month.head()

,shop_id,item_id,item_cnt_month
2263493,2,31,1
2263494,2,486,1
2263495,2,486,2
2263496,2,787,1
2263497,2,794,1


In [38]:
shop_pre_month.columns = ['shop_id', 'item_id', 'item_cnt_prev_month']

In [39]:
shop_pre_month.head()

,shop_id,item_id,item_cnt_prev_month
2263493,2,31,1
2263494,2,486,1
2263495,2,486,2
2263496,2,787,1
2263497,2,794,1


In [41]:
train.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month,item_cnt_mean
0,0,0,32,221.0,1,2013,4,2.666667
1,0,0,32,221.0,3,2013,2,2.666667
2,0,0,33,347.0,1,2013,2,1.500000
3,0,0,33,347.0,3,2013,1,1.500000
4,0,0,35,247.0,1,2013,1,1.875000


In [42]:
train = pd.merge(train, shop_pre_month, how="left", on=['shop_id', 'item_id']).fillna(0)

In [43]:
train.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month,item_cnt_mean,item_cnt_prev_month
0,0,0,32,221.0,1,2013,4,2.666667,0.0
1,0,0,32,221.0,3,2013,2,2.666667,0.0
2,0,0,33,347.0,1,2013,2,1.500000,0.0
3,0,0,33,347.0,3,2013,1,1.500000,0.0
4,0,0,35,247.0,1,2013,1,1.875000,0.0


In [44]:
train = pd.merge(train, items, how='left', on=['item_id'])
train = pd.merge(train, item_categories, how='left', on=['item_category_id'])
train = pd.merge(train, shops, how='left', on=['shop_id'])
train.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month,item_cnt_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,221.0,1,2013,4,2.666667,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,32,221.0,3,2013,2,2.666667,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
2,0,0,33,347.0,1,2013,2,1.500000,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
3,0,0,33,347.0,3,2013,1,1.500000,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
4,0,0,35,247.0,1,2013,1,1.875000,0.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"


In [45]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34

In [46]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num
0,0,5,5037,11,2015,34
1,1,5,5320,11,2015,34
2,2,5,5233,11,2015,34
3,3,5,5232,11,2015,34
4,4,5,5268,11,2015,34


In [47]:
test = pd.merge(test, shop_item_mean, how='left', on=['shop_id', 'item_id']).fillna(0)

In [48]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean
0,0,5,5037,11,2015,34,1.083333
1,1,5,5320,11,2015,34,0.000000
2,2,5,5233,11,2015,34,1.250000
3,3,5,5232,11,2015,34,1.000000
4,4,5,5268,11,2015,34,0.000000


In [49]:
test = pd.merge(test, shop_pre_month, how='left', on=['shop_id', 'item_id']).fillna(0)

In [50]:
test = pd.merge(test, items, how='left', on=['item_id']).fillna(0)

In [51]:
test = pd.merge(test, item_categories, how='left', on=['item_category_id']).fillna(0)

In [52]:
test = pd.merge(test, shops, how='left', on=['shop_id']).fillna(0)

In [53]:
test['item_cnt_month'] = 0

In [54]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.083333,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0
2,2,5,5233,11,2015,34,1.250000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0


In [55]:
from sklearn.preprocessing import LabelEncoder

In [57]:
for item in ['shop_name', 'item_name', 'item_category_name']:
    lbl = LabelEncoder()
    lbl.fit(list(train[item].unique()) + list(test[item].unique()))
    train[item] = lbl.transform(train[item].astype(str))
    test[item] = lbl.transform(test[item].astype(str))
    print(item)


shop_name
item_name
item_category_name


In [59]:
final_train = train.drop('item_price', axis=1)

In [75]:
final_train.columns

Index(['date_block_num', 'shop_id', 'item_id', 'month', 'year',
       'item_cnt_month', 'item_cnt_mean', 'item_cnt_prev_month', 'item_name',
       'item_category_id', 'item_category_name', 'shop_name'],
      dtype='object')

In [76]:
test.columns

Index(['ID', 'shop_id', 'item_id', 'month', 'year', 'date_block_num',
       'item_cnt_mean', 'item_cnt_prev_month', 'item_name', 'item_category_id',
       'item_category_name', 'shop_name', 'item_cnt_month'],
      dtype='object')

In [77]:
col = [c for c in final_train.columns if c not in ['item_cnt_month']]
final_train = final_train[final_train['date_block_num'] < 33]
train_y = np.log1p(final_train['item_cnt_month'].clip(0, 20))
train_data = final_train[col]

val_data = final_train[final_train['date_block_num'] == 33]
val_y = np.log1p(val_data['item_cnt_month'].clip(0, 20))
val_data = val_data[col]

In [78]:
from sklearn.ensemble import ExtraTreesRegressor

In [79]:
extra_trees = ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=20, random_state=0)

In [81]:
extra_trees.fit(train_data, train_y)

ExtraTreesRegressor(max_depth=20, n_estimators=25, n_jobs=-1, random_state=0)

In [82]:
from sklearn.metrics import mean_squared_error

val_rmse = np.sqrt(mean_squared_error(extra_trees.predict(val_data).clip(0, 10), val_y))

In [83]:
val_rmse

0.20689583003241868

In [84]:
extra_trees.fit(final_train[col], final_train['item_cnt_month'])

ExtraTreesRegressor(max_depth=20, n_estimators=25, n_jobs=-1, random_state=0)

In [85]:
predictions = extra_trees.predict(test[col]).clip(0, 20)

In [86]:
predictions

array([0.91036112, 0.08809852, 1.0720075 , ..., 0.96188149, 0.        ,
       1.12483096])

In [87]:
prediction_df = pd.DataFrame(test['ID'], columns=['ID'])
prediction_df['item_cnt_month'] = predictions
prediction_df.to_csv('submission.csv', index=False)
prediction_df.head(10)

,ID,item_cnt_month
0,0,0.910361
1,1,0.088099
2,2,1.072008
3,3,0.892103
4,4,0.161983
5,5,1.075433
6,6,1.077358
7,6,1.077358
8,7,1.237491
9,8,1.299563
